In [37]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from IPython.display import HTML
import time

In [38]:
browser = Browser('chrome')
base_url = 'https://www.indeed.com'
url = 'https://www.indeed.com/jobs?q=junior+data+analyst&l=&from=searchOnHP&vjk=5e72f39a23380519'
browser.visit(url)

In [39]:
#html = browser.html
#soup = BeautifulSoup(html,'html.parser')

In [40]:
#results = soup.find('ul', class_='jobsearch-ResultsList css-0')
#jobs = results.find_all('td', class_='resultContent')

In [41]:
job_list = []
exclude_words = ["a", "an", "the", "at", "by", "for", "in", "of", "on", "to", "with", "and", "but", "or", "so", "he", "she", "it", "they", "them", "you", "your", "we", "us", "our", "i", "me", "my", "mine", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "do", "does", "did", "will", "would", "should", "could", "may", "might", "must", "thing", "stuff", "something", "anything", "everything", "nothing", "one", "ones", "some", "many", "few","their"]


In [42]:
def count_words(text, exclude=[]):
    words = text.split()
    words = [word.lower() for word in words if word.lower() not in exclude]
    word_counts = {word: words.count(word) for word in set(words)}
    return word_counts


In [43]:
# Modify the loop to click on the job listing and count the words
while True:
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    results = soup.find('ul', class_='jobsearch-ResultsList css-0')
    jobs = results.find_all('td', class_='resultContent')
    
    for job in jobs:

        title = job.div.h2.a.text
        company = job.find('span', class_='companyName').text
        location = job.find('div', class_='companyLocation').text
        estimate = job.find('span', class_='estimated-salary')
        if estimate is not None:
            salary = estimate.text
        else:
            salary = 'N/A'

        link = job.find('a')['href']
        absolute_link = urljoin(base_url, link)        
        clickable_link = '<a href="' + absolute_link + '">Link</a>'
        
        # Click on the job listing and wait for the job description to appear
        browser.visit(absolute_link)
        time.sleep(15)

        
        # Extract the job description text and count the words
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        job_description = soup.find('div', class_='jobsearch-jobDescriptionText')
        if job_description is not None:
            text = job_description.get_text()
            word_count = count_words(text, exclude=exclude_words)
            most_common_word = max(word_count, key=word_count.get)
        else:
            word_count = {}
            most_common_word = 'N/A'
        browser.back()

        job_dict={
            'Title':title,
            'Company':company,
            'Location':location,
            'Salary':salary,
            'Link':clickable_link,
            'Word count':word_count,
            'Most common word':most_common_word
        }
        job_list.append(job_dict)
        
               
    button = browser.find_by_css('a[data-testid="pagination-page-next"]')
    if not button:
        break
        
    button.click()
    time.sleep(20)


In [44]:
df = pd.DataFrame(job_list)

In [45]:
df.head(25)

,Title,Company,Location,Salary,Link,Word count,Most common word
0,Junior Data Analyst,"Zartico, Inc.",Remote,Estimated $68.1K - $86.2K a year,"<a href=""https://www.indeed.com/rc/clk?jk=2ef2...","{'join': 3, 'team,': 1, 'industry': 1, 'job': ...",data
1,Junior Business Analyst,Janus Henderson,"Hybrid remote in Denver, CO",N/A,"<a href=""https://www.indeed.com/rc/clk?jk=7fe4...","{'architects': 1, 'join': 2, 'rewards': 1, 'ti...",work
2,Enterprise Data Analyst Junior,STM Consulting.INC,"Denver, CO 80204 (Auraria area)",N/A,"<a href=""https://www.indeed.com/company/STM-Co...","{'reports,': 1, 'portfolio.': 1, 'websites': 2...",·
3,Junior Business Analyst,aezion inc,Remote,N/A,"<a href=""https://www.indeed.com/company/Aezion...","{'junior': 2, '3': 1, 'teams': 1, 'contract': ...",business
4,Junior Data Analyst,Affiliated Monitoring,"Union, NJ 07083",N/A,"<a href=""https://www.indeed.com/pagead/clk?mo=...","{'post': 1, 'administration,': 1, 'entrepreneu...",data
5,Junior Data Analyst - W2,American Unit,"Golden, CO 80401",N/A,"<a href=""https://www.indeed.com/company/Americ...","{'junior': 1, 'data': 1, 'contract': 1, 'candi...",experience
6,Junior Data Analyst,Aptive,Remote+1 location,N/A,"<a href=""https://www.indeed.com/rc/clk?jk=3825...","{'reports,': 1, 'organizations': 1, 'join': 1,...",data
7,Junior Data Analyst,General Dynamics Information Technology,Remote in Virginia+1 location,Estimated $93.7K - $119K a year,"<a href=""https://www.indeed.com/rc/clk?jk=c2d2...","{'you'll': 2, 'equivalent': 1, 'junior': 1, 'm...",data
8,Junior Business Analyst,BPA Services,Remote,N/A,"<a href=""https://www.indeed.com/company/Busine...","{'improvements': 1, 'time': 1, 'junior': 1, 'j...",as
9,Junior Data Analyst,Alpha Facilities Solutions,"San Antonio, TX 78261",Estimated $74.5K - $94.3K a year,"<a href=""https://www.indeed.com/rc/clk?jk=37a9...","{'deliverables': 1, 'senior': 1, 'reason': 1, ...",business


In [48]:
df.to_csv('../Resources/output.csv', index=False)